In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9MB 114kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 4.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595677 sha256=a06828ad6adf6743c63e3fee0fc7c731c11bc0e08f3a0cec369f66f6b982b510
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 4.3MB/s 
     |████████████████████████████████| 757kB 4.2MB/s 
     |████████████████████████████████| 3.0MB 28.4MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nsdfqvia
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nsdfqvia
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=22f8facbc4ccffbb7763bde258839976ecc894c33bb2f4ac6543bbabdb3c09e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-aaorvxt3/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1PWcT5ucCWPPefzJVLBjC1KszKamn_vyR" -O datatest.tsv
!wget "https://drive.google.com/uc?export=download&id=13xxABWzRT0mjoPEKWApezhX96XPH58sk" -O datatrain.tsv

--2021-05-18 01:42:37--  https://drive.google.com/uc?export=download&id=1PWcT5ucCWPPefzJVLBjC1KszKamn_vyR
Resolving drive.google.com (drive.google.com)... 64.233.189.102, 64.233.189.139, 64.233.189.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6fdbn3i0sbksgcaabbo5bvb0c5utqnpk/1621302150000/16667090296722432029/*/1PWcT5ucCWPPefzJVLBjC1KszKamn_vyR?e=download [following]
--2021-05-18 01:42:38--  https://doc-00-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6fdbn3i0sbksgcaabbo5bvb0c5utqnpk/1621302150000/16667090296722432029/*/1PWcT5ucCWPPefzJVLBjC1KszKamn_vyR?e=download
Resolving doc-00-8g-docs.googleusercontent.com (doc-00-8g-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c01::84
Connecting to doc-00-8g-docs.googleusercontent.com (doc-00

In [ ]:
dataset_train = nlp.data.TSVDataset("datatrain.tsv")

In [ ]:

dataset_test = nlp.data.TSVDataset("datatest.tsv")

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(int(float(i[label_idx]))) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 50
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
print(dataset_train[:5])

[['징글징글 해 아직도 교회', '0.0'], ['노래방 띄어 앉으면 된다 하고 . 공연 하고 있고 . , 뉴스 마스크 없이 하고 가수 . 트로트 무수 히 노래 * 랄 설교 자만 마스크 흐 하라 거 ? 공정하냐 ? 도대체', '0.0'], ['기사 수준 이하 생각 됨 . 어거지 어거지 ... 방송 드라마 뿐 아니라 TV 나오는 모든 사람 마스크 안 쓰고 나오면 ... 명단 만들어서 꼭 기사 올려요 . ㅋㅋㅋ', '0.0'], ['전세계 차 대유행 데 우리나라 종식 된줄 아는 사람 때문 너무 위험합니다', '0.0'], ['클럽 . 한강 . 술집 . 운동 하는 곳 . 촬영 . 안 써요 ... 교회 욕 !~~~', '0.0']]


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.712497889995575 train acc 0.5

epoch 1 train acc 0.7163461538461539


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.98125


...


epoch 8 test acc 0.95625


epoch 9 batch id 1 loss 0.0032758815214037895 train acc 1.0

epoch 9 train acc 0.9951923076923077



epoch 9 test acc 0.9625


epoch 10 batch id 1 loss 0.004528840538114309 train acc 1.0

epoch 10 train acc 0.9975961538461539



epoch 10 test acc 0.9625


In [ ]:

import pandas as pd

def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = []
  emo_list = []
  for c in comment:
    commnetslist.append( [c, 5] )
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):#enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    out = model(token_ids, valid_length, segment_ids)
    #print("out one:",out)
  
    for e in out:
      #print(e)
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list


'\nunseen_test = pd.DataFrame([["기독교 기적 면 많다", 1], ["신랑 정신차리도록 내조 확실히 하도록.",1]\n                            , ["가나다 라마바 사아자 고마워",1]], columns = [[\'댓글\', \'감성\']]) \nunseen_values = unseen_test.values\nprint(unseen_values)\ntest_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False) \ntest_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)\n\nfor batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):\n    token_ids = token_ids.long().to(device) \n    segment_ids = segment_ids.long().to(device) \n    valid_length= valid_length \n    label = label.long().to(device)\n    print("label:", label)\n    out = model(token_ids, valid_length, segment_ids) \n    print("out:",out)\n'

In [ ]:
from google.colab import drive
import json

tlist = [] # database table name list
for i in range(1,6):
    t = "daum"
    tlist.append(t+"after"+str(i))
    tlist.append(t+"before"+str(i))
for i in range(1,6):
    t = "naver"
    tlist.append(t+"after"+str(i))
    tlist.append(t+"before"+str(i))

drive.mount("/content/drive")
path = "/content/drive/My Drive/ColabNotebooks/jsonComment/" # comment  json file directory path
for tablename in tlist:
    
    with open(path+tablename+'-dict.json', encoding="utf-8") as json_file:
        data = json.load(json_file) # load json file
    
    for date in data.keys(): 
        print(">>",date,"/",tablename)
        
        day_article = data[date]
        for article in day_article.keys():
            if len(day_article[article][0])>100: print("카운팅:",len(day_article[article][0]))
            comments = day_article[article][0]
            if len(comments)==1: continue
            
            comments.remove("")
            
            emotion = getSentimentValue(comments, tok, max_len, batch_size, device)
            
            data[date][article][-1].extend(emotion)
            #data[date][article][-1].append(emotion) # dictionary value append sentiment
    print(tablename, "--comment save")
    result_path = "/content/drive/My Drive/ColabNotebooks/predictComment/" #  result save json path
    #drive.mount("/Colab Notebooks/predictComment")
    with open(result_path+"finish-"+tablename+"-dict.json", "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, indent="\t",ensure_ascii = False)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>> 2020/7/31 / daumafter1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


카운팅: 121
카운팅: 559
카운팅: 1286
>> 2020/8/14 / daumafter1
>> 2020/8/3 / daumafter1
>> 2020/8/11 / daumafter1
카운팅: 147
카운팅: 152
>> 2020/8/13 / daumafter1
카운팅: 1164
카운팅: 183
>> 2020/8/1 / daumafter1
카운팅: 108
카운팅: 1211
카운팅: 500
카운팅: 253
카운팅: 217
카운팅: 110
카운팅: 1152
카운팅: 319
카운팅: 405
카운팅: 173
카운팅: 413
카운팅: 1128
>> 2020/8/17 / daumafter1
>> 2020/8/16 / daumafter1
카운팅: 384
>> 2020/8/18 / daumafter1
카운팅: 677
카운팅: 216
카운팅: 986
카운팅: 132
카운팅: 241
카운팅: 401
카운팅: 104
카운팅: 104
카운팅: 271
>> 2020/7/30 / daumafter1
카운팅: 1071
카운팅: 108
>> 2020/7/29 / daumafter1
카운팅: 1119
>> 2020/7/28 / daumafter1
>> 2020/8/2 / daumafter1
카운팅: 442
>> 2020/7/22 / daumafter1
>> 2020/7/23 / daumafter1
>> 2020/8/12 / daumafter1
카운팅: 419
>> 2020/7/18 / daumafter1
카운팅: 743
카운팅: 846
>> 2020/7/17 / daumafter1
>> 2020/7/13 / daumafter1
카운팅: 159
카운팅: 1039
카운팅: 109
카운팅: 870
카운팅: 418
카운팅: 822
카운팅: 1034
카운팅: 266
카운팅: 345
카운팅: 1041
카운팅: 1010
>> 2020/6/22 / daumafter1
카운팅: 149
>> 2020/7/16 / daumafter1
>> 2020/8/5 / daumafter1
>> 2020/7/14 / 

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line

카운팅: 413
>> 2020/5/6 / daumafter3
>> 2020/5/28 / daumafter3
>> 2020/4/28 / daumafter3
>> 2020/4/8 / daumafter3
>> 2020/4/16 / daumafter3
>> 2020/3/18 / daumafter3
>> 2020/5/16 / daumafter3
>> 2020/3/25 / daumafter3
>> 2020/4/3 / daumafter3
>> 2020/2/25 / daumafter3
카운팅: 106
>> 2020/3/19 / daumafter3
카운팅: 461
>> 2020/4/29 / daumafter3
>> 2020/2/24 / daumafter3
>> 2020/4/2 / daumafter3
카운팅: 1014
>> 2020/2/29 / daumafter3
>> 2020/4/7 / daumafter3
>> 2020/5/3 / daumafter3
>> 2020/3/11 / daumafter3
카운팅: 193
>> 2020/3/6 / daumafter3
>> 2020/3/21 / daumafter3
카운팅: 111
>> 2020/3/20 / daumafter3
>> 2020/6/30 / daumafter3
>> 2020/5/12 / daumafter3
>> 2020/2/27 / daumafter3
>> 2020/4/15 / daumafter3
>> 2020/3/5 / daumafter3
>> 2020/4/10 / daumafter3
>> 2020/3/9 / daumafter3
>> 2020/3/23 / daumafter3
>> 2020/2/20 / daumafter3
>> 2020/2/22 / daumafter3
>> 2020/7/2 / daumafter3
>> 2020/5/15 / daumafter3
>> 2020/3/4 / daumafter3
>> 2020/2/23 / daumafter3
>> 2020/3/17 / daumafter3
>> 2020/4/4 / daumaf